In [2]:
# set up Python environment: numpy for numerical routines, and matplotlib for plotting
import numpy as np
import matplotlib.pyplot as plt
from subprocess import call
from shutil import rmtree
import gdown
import sys
import os

caffe_root = os.environ['CAFFE_ROOT']
project_root = os.getcwd()

import caffe

In [3]:
# display plots in this notebook
%matplotlib inline
# set display defaults
plt.rcParams['figure.figsize'] = (10, 10)        # large images
plt.rcParams['image.interpolation'] = 'nearest'  # don't interpolate: show square pixels
plt.rcParams['image.cmap'] = 'gray'  # use grayscale output rather than a (potentially misleading) color heatmap

In [5]:
prototxt_url = 'https://drive.google.com/open?id=1-kT2Vnr_TgUcQa7cz6acXW7n5GWRtFyF'
prototxt_name = 'trailnet_color_o3.prototxt'

caffemodel_url = 'https://drive.google.com/open?id=1fWCQ5ZsY_KziwqGWeLJuUl6IlAh98dSi'
caffemodel_name = 'trailnet_3class_real_vr_mix_color.caffemodel'

if not os.path.isfile(caffemodel_name):
    print('Downloading pre-trained TrailNet model...')
    gdown.download(caffemodel_url, output=caffemodel_name, quiet=False)
if not os.path.isfile(prototxt_name):
    print('Downloading pre-trained TrailNet prototxt...')
    gdown.download(prototxt_url, output=prototxt_name, quiet=False)

print('Finished preparing the model.')             

Finished preparing the model.


In [ ]:
def train_split(split_num, cwd, solverpath):
    caffe_path = os.path.join(CAFFE_ROOT, "build/tools/caffe")
    relative_model_path = ("models/bvlc_reference_caffenet/")
    weights_path = os.path.join(CAFFE_ROOT, relative_model_path,
                                "bvlc_reference_caffenet.caffemodel")

    # fine tune from bvlc reference caffe model
    call([caffe_path, "train", "-solver", solverpath,
          "-weights", weights_path, "-gpu", str(GPU)])


def evaluate_results(split):
    # create the confusion matrix and link the misclassified images
    call(["./CaffeNetAnalysis/CaffeNetAnalysisMain",
          "../splits/test{0}.txt".format(split),
          os.path.join(os.path.abspath("../images/"), ""),
          "../caffe_data/deploy.prototxt",
          "../results/{0}/snapshots/_iter_10000.caffemodel".format(split),
          "../classid.txt",
          "../results/{0}/".format(split), str(GPU)])

def export_np_mat_with_header(mean_mat, std_dev_mat, file_to_copy_header,
                              export_filename, skip_header=0, skip_footer=0):
    with open(file_to_copy_header, 'r') as f:
        lines = f.readlines()
        header = lines[:skip_header]
        lines = lines[skip_header:len(lines) - skip_footer]
        first_column = [l.split(';')[0] for l in lines]
    with open(export_filename, 'w') as f:
        if header:
            f.writelines(header)  # copy header
        for i in range(len(first_column)):
            if len(mean_mat.shape) == 1:
                mean_stddev = [("{:.3f}".format(mean_mat[i]),
                                "{:.3f}".format(std_dev_mat[i]))]
            else:
                mean_stddev = zip(map("{:.3f}".format, mean_mat[i]),
                                  map("{:.3f}".format, std_dev_mat[i]))
            mean_mat_line = "; ".join([" +- ".join(m) for m in mean_stddev])
            f.write(first_column[i] + "; " + mean_mat_line + "; \n")

def evaluate_mean():
    confusion_mats = []
    accuracy_mats = []
    for i in range(1):
        confusion_mat_path = "../results/{0}/confusion_matrix.csv".format(i)
        accuracy_mat_path = "../results/{0}/accuracy.csv".format(i)
        confusion_mats.append(np.genfromtxt(confusion_mat_path,
                                            usecols=list(range(1, 4)),
                                            delimiter=';', skip_header=1,
                                            skip_footer=1))
        accuracy_mats.append(np.genfromtxt(accuracy_mat_path,
                                           usecols=(1,),
                                           delimiter=';'))
    export_np_mat_with_header(np.mean(confusion_mats, axis=0),
                              np.std(confusion_mats, axis=0),
                              "../results/0/confusion_matrix.csv",
                              "../results/mean_confusion_matrix.csv", 1, 1)
    export_np_mat_with_header(np.mean(accuracy_mats, axis=0),
                              np.std(accuracy_mats, axis=0),
                              "../results/0/accuracy.csv",
                              "../results/mean_accuracy_matrix.csv")


def check_if_training_files_exist(storage_dir):
    if not os.path.isdir(storage_dir):
        return
    print("It seems there already exist files from a previous"
          "training. Delete all files in folder results? y/n")
    while True:
        inp = str(input())
        if inp.lower() == 'y':
            for i in range(1):
                rmtree("../results/".format(i))
            break
        elif inp.lower() == 'n':
            exit()


if __name__ == "__main__":
    for i in range(1):  # train the 5 splits
        cwd = os.getcwd()
        evaluate_results(i)
    evaluate_mean()
